## Anaconda Euler

In [1]:
import io
import itertools
import functools
import requests

import numpy as np
from scipy import linalg, special, optimize
import pandas as pd
import sympy
import numba
import num2words

class Prime():
    def __init__(self, n=10):
        self.prime_list = np.array([2])
        self.prime_generate(n)
        
    def prime_generate(self, n):
        """ Input n>=6, Returns a array of primes, 2 <= p < n """
        if self.prime_list[-1] < n:
            sieve = np.ones(n // 3 + (n % 6 == 2), dtype=np.bool)
            for i in range(1, int(n ** 0.5) // 3 + 1):
                if sieve[i]:
                    k = 3 * i + 1 | 1
                    sieve[k * k // 3::2 * k] = False
                    sieve[k * (k - 2 * (i & 1) + 4) // 3::2 * k] = False
            self.prime_list = np.r_[2, 3, ((3 * np.nonzero(sieve)[0][1:] + 1) | 1)]
    
    def primerange(self, n):
        self.prime_generate(n)
        return(self.prime_list[:np.searchsorted(self.prime_list, n)])
    
    def prime(self, n):
        self.prime_generate(int(np.max(2 * n * np.log(n))))
        return(self.prime_list[n - 1])
    
    def isprime(self, n):
        n_key = np.searchsorted(self.prime_list, n)
        if n_key >= len(self.prime_list):
            self.prime_generate(2 * n)
        return(self.prime_list[n_key] == n)
    
prime = Prime(int(10 ** 9))

In [2]:
%%time
## Problem 1: Multiples of 3 and 5

n_list = [3, 5]
n_max = 1000

print(np.unique(np.concatenate([np.arange(0, n_max, i) for i in n_list])).sum())

233168
CPU times: user 209 µs, sys: 53 µs, total: 262 µs
Wall time: 266 µs


In [3]:
%%time
## Problem 2: Even Fibonacci numbers

n_max = 4e6

phi = (1 + np.sqrt(5)) / 2
def fibonacci(n, n_init=0):
    n_list = n_init + np.arange(n)
    return(((phi ** n_list - (1 - phi) ** n_list) / np.sqrt(5)).astype('int'))

i_max = int(np.log(4e6) / np.log(phi)) + 1
print(sum([i for i in fibonacci(i_max, n_init=2) if (i < n_max) & (i % 2 == 0)]))

4613732
CPU times: user 458 µs, sys: 193 µs, total: 651 µs
Wall time: 499 µs


In [4]:
%%time
## Problem 3: Largest prime factor

n = 600851475143

print(max(sympy.factorint(n).keys()))

6857
CPU times: user 524 µs, sys: 127 µs, total: 651 µs
Wall time: 555 µs


In [5]:
%%time
## Problem 4: Largest palindrome product

n_min = 100
n_max = 1000

S = pd.Series(np.unique(np.outer(np.arange(n_min, n_max), np.arange(n_min, n_max))).astype("str"))
print(int(S[S == S.str[::-1]].iloc[-1]))

906609
CPU times: user 231 ms, sys: 34.9 ms, total: 266 ms
Wall time: 264 ms


In [6]:
%%time
## Problem 5: Smallest multiple

n_list = range(1, 21)

S = pd.concat([pd.Series(sympy.factorint(i)) for i in n_list], axis=1).fillna(0).max(axis=1).astype("int")
print(np.prod(S.index.values ** S.values))

232792560
CPU times: user 16.6 ms, sys: 2.74 ms, total: 19.3 ms
Wall time: 18.6 ms


In [7]:
%%time
## Problem 6: Sum square difference

n_list = 1 + np.arange(100)

print(sum(n_list) ** 2 - sum(n_list ** 2))

25164150
CPU times: user 416 µs, sys: 392 µs, total: 808 µs
Wall time: 462 µs


In [8]:
%%time
## Problem 7: 10001st prime

n = 10001

print(prime.prime(n))

104743
CPU times: user 117 µs, sys: 65 µs, total: 182 µs
Wall time: 150 µs


In [9]:
%%time
## Problem 8: Largest product in a series

n = 13
s = '''73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450'''.replace("\n", "")

print(pd.DataFrame([list(s[i:(i+13)]) for i in range(len(s) - n)]).astype("int").prod(axis=1).max())

23514624000
CPU times: user 5.97 ms, sys: 1.38 ms, total: 7.35 ms
Wall time: 6.21 ms


In [10]:
%%time
## Problem 9: Special Pythagorean triplet

n = 1000

a_list = np.arange(n / 2)
b_list = np.arange(n / 2)
c_list = n - np.add.outer(a_list, b_list)

a, b = np.where(c_list ** 2 == np.add.outer(a_list ** 2, b_list ** 2))[0]
print(a * b * (n - a - b))

31875000
CPU times: user 4.4 ms, sys: 3.26 ms, total: 7.67 ms
Wall time: 5.91 ms


In [11]:
%%time
## Problem 10: Summation of primes

n_max = int(2e6)

print(sum(prime.primerange(n_max)))

142913828922
CPU times: user 16.1 ms, sys: 423 µs, total: 16.5 ms
Wall time: 16.2 ms


In [12]:
%%time
## Problem 11: Largest product in a grid

n = 4
A = np.loadtxt(io.StringIO('''08 02 22 97 38 15 00 40 00 75 04 05 07 78 52 12 50 77 91 08
49 49 99 40 17 81 18 57 60 87 17 40 98 43 69 48 04 56 62 00
81 49 31 73 55 79 14 29 93 71 40 67 53 88 30 03 49 13 36 65
52 70 95 23 04 60 11 42 69 24 68 56 01 32 56 71 37 02 36 91
22 31 16 71 51 67 63 89 41 92 36 54 22 40 40 28 66 33 13 80
24 47 32 60 99 03 45 02 44 75 33 53 78 36 84 20 35 17 12 50
32 98 81 28 64 23 67 10 26 38 40 67 59 54 70 66 18 38 64 70
67 26 20 68 02 62 12 20 95 63 94 39 63 08 40 91 66 49 94 21
24 55 58 05 66 73 99 26 97 17 78 78 96 83 14 88 34 89 63 72
21 36 23 09 75 00 76 44 20 45 35 14 00 61 33 97 34 31 33 95
78 17 53 28 22 75 31 67 15 94 03 80 04 62 16 14 09 53 56 92
16 39 05 42 96 35 31 47 55 58 88 24 00 17 54 24 36 29 85 57
86 56 00 48 35 71 89 07 05 44 44 37 44 60 21 58 51 54 17 58
19 80 81 68 05 94 47 69 28 73 92 13 86 52 17 77 04 89 55 40
04 52 08 83 97 35 99 16 07 97 57 32 16 26 26 79 33 27 98 66
88 36 68 87 57 62 20 72 03 46 33 67 46 55 12 32 63 93 53 69
04 42 16 73 38 25 39 11 24 94 72 18 08 46 29 32 40 62 76 36
20 69 36 41 72 30 23 88 34 62 99 69 82 67 59 85 74 04 36 16
20 73 35 29 78 31 90 01 74 31 49 71 48 86 81 16 23 57 05 54
01 70 54 71 83 51 54 69 16 92 33 48 61 43 52 01 89 19 67 48'''), dtype = 'int')

A_pad = np.pad(A, [[1, 1], [1, 1]], mode="constant", constant_values=0)
max_v = np.max(functools.reduce(np.multiply, [A_pad[i:-(n - i)] for i in range(n)]))
max_h = np.max(functools.reduce(np.multiply, [A_pad[:, i:-(n - i)] for i in range(n)]))
max_d = np.max(functools.reduce(np.multiply, [A_pad[i:-(n - i), i:-(n - i)] for i in range(n)]))
max_ad = np.max(functools.reduce(np.multiply, [A_pad[(n - i):-(i + 1), i:-(n - i)] for i in range(n)]))
print(max(max_v, max_h, max_d, max_ad))

70600674
CPU times: user 1.67 ms, sys: 774 µs, total: 2.45 ms
Wall time: 1.85 ms


In [13]:
%%time
## Problem 12: Highly divisible triangular number

n = 500

i_val = 0
for i in itertools.count():
    i_val += i
    if len(sympy.divisors(i_val)) > 500:
        break
print(i_val)

76576500
CPU times: user 1.06 s, sys: 8.47 ms, total: 1.07 s
Wall time: 1.07 s


In [14]:
%%time
## Problem 13: Large sum

n = 10
n_list = np.loadtxt(io.StringIO('''37107287533902102798797998220837590246510135740250
46376937677490009712648124896970078050417018260538
74324986199524741059474233309513058123726617309629
91942213363574161572522430563301811072406154908250
23067588207539346171171980310421047513778063246676
89261670696623633820136378418383684178734361726757
28112879812849979408065481931592621691275889832738
44274228917432520321923589422876796487670272189318
47451445736001306439091167216856844588711603153276
70386486105843025439939619828917593665686757934951
62176457141856560629502157223196586755079324193331
64906352462741904929101432445813822663347944758178
92575867718337217661963751590579239728245598838407
58203565325359399008402633568948830189458628227828
80181199384826282014278194139940567587151170094390
35398664372827112653829987240784473053190104293586
86515506006295864861532075273371959191420517255829
71693888707715466499115593487603532921714970056938
54370070576826684624621495650076471787294438377604
53282654108756828443191190634694037855217779295145
36123272525000296071075082563815656710885258350721
45876576172410976447339110607218265236877223636045
17423706905851860660448207621209813287860733969412
81142660418086830619328460811191061556940512689692
51934325451728388641918047049293215058642563049483
62467221648435076201727918039944693004732956340691
15732444386908125794514089057706229429197107928209
55037687525678773091862540744969844508330393682126
18336384825330154686196124348767681297534375946515
80386287592878490201521685554828717201219257766954
78182833757993103614740356856449095527097864797581
16726320100436897842553539920931837441497806860984
48403098129077791799088218795327364475675590848030
87086987551392711854517078544161852424320693150332
59959406895756536782107074926966537676326235447210
69793950679652694742597709739166693763042633987085
41052684708299085211399427365734116182760315001271
65378607361501080857009149939512557028198746004375
35829035317434717326932123578154982629742552737307
94953759765105305946966067683156574377167401875275
88902802571733229619176668713819931811048770190271
25267680276078003013678680992525463401061632866526
36270218540497705585629946580636237993140746255962
24074486908231174977792365466257246923322810917141
91430288197103288597806669760892938638285025333403
34413065578016127815921815005561868836468420090470
23053081172816430487623791969842487255036638784583
11487696932154902810424020138335124462181441773470
63783299490636259666498587618221225225512486764533
67720186971698544312419572409913959008952310058822
95548255300263520781532296796249481641953868218774
76085327132285723110424803456124867697064507995236
37774242535411291684276865538926205024910326572967
23701913275725675285653248258265463092207058596522
29798860272258331913126375147341994889534765745501
18495701454879288984856827726077713721403798879715
38298203783031473527721580348144513491373226651381
34829543829199918180278916522431027392251122869539
40957953066405232632538044100059654939159879593635
29746152185502371307642255121183693803580388584903
41698116222072977186158236678424689157993532961922
62467957194401269043877107275048102390895523597457
23189706772547915061505504953922979530901129967519
86188088225875314529584099251203829009407770775672
11306739708304724483816533873502340845647058077308
82959174767140363198008187129011875491310547126581
97623331044818386269515456334926366572897563400500
42846280183517070527831839425882145521227251250327
55121603546981200581762165212827652751691296897789
32238195734329339946437501907836945765883352399886
75506164965184775180738168837861091527357929701337
62177842752192623401942399639168044983993173312731
32924185707147349566916674687634660915035914677504
99518671430235219628894890102423325116913619626622
73267460800591547471830798392868535206946944540724
76841822524674417161514036427982273348055556214818
97142617910342598647204516893989422179826088076852
87783646182799346313767754307809363333018982642090
10848802521674670883215120185883543223812876952786
71329612474782464538636993009049310363619763878039
62184073572399794223406235393808339651327408011116
66627891981488087797941876876144230030984490851411
60661826293682836764744779239180335110989069790714
85786944089552990653640447425576083659976645795096
66024396409905389607120198219976047599490197230297
64913982680032973156037120041377903785566085089252
16730939319872750275468906903707539413042652315011
94809377245048795150954100921645863754710598436791
78639167021187492431995700641917969777599028300699
15368713711936614952811305876380278410754449733078
40789923115535562561142322423255033685442488917353
44889911501440648020369068063960672322193204149535
41503128880339536053299340368006977710650566631954
81234880673210146739058568557934581403627822703280
82616570773948327592232845941706525094512325230608
22918802058777319719839450180888072429661980811197
77158542502016545090413245809786882778948721859617
72107838435069186155435662884062257473692284509516
20849603980134001723930671666823555245252804609722
53503534226472524250874054075591789781264330331690'''))

out = sum(n_list)
print(int(out / 10 ** int(np.log10(out) - n + 1)))

5537376230
CPU times: user 1.22 ms, sys: 65 µs, total: 1.28 ms
Wall time: 1.27 ms


In [15]:
%%time
## Problem 14: Longest Collatz sequence

n_min = 2
n_max = int(1e6)

chain_val_dict = {}
chain_len_dict = {1: 0}
def chain(x):
    if x != 1:
        x_next = chain_val_dict.get(x)
        if x_next is None:
            if x % 2 == 1:
                x_next = 3 * x + 1
            else:
                x_next = x // 2
            chain_val_dict[x] = x_next
            chain(x_next)
        chain_len_dict[x] = chain_len_dict[x_next] + 1

[chain(i) for i in range(n_min, n_max)]
print(pd.Series(chain_len_dict)[:n_max].argmax())

837799
CPU times: user 6.22 s, sys: 204 ms, total: 6.43 s
Wall time: 6.42 s


In [16]:
%%time
## Problem 15: Lattice paths

n = 20

print(special.comb(2 * n, n, exact=True))

137846528820
CPU times: user 301 µs, sys: 244 µs, total: 545 µs
Wall time: 305 µs


In [17]:
%%time
## Problem 16: Power digit sum

n = 2 ** 1000

print(np.array(list(str(n)), dtype="int").sum())

1366
CPU times: user 359 µs, sys: 245 µs, total: 604 µs
Wall time: 454 µs


In [18]:
%%time
## Problem 17: Number letter counts

n_max = 1000

print(len("".join([num2words.num2words(i) for i in 1 + np.arange(n_max)]).replace(" ", "").replace("-", "")))

21124
CPU times: user 1.76 s, sys: 6.34 ms, total: 1.77 s
Wall time: 1.76 s


In [19]:
%%time
## Problem 18: Maximum path sum I

A = pd.read_fwf(io.StringIO('''75
95 64
17 47 82
18 35 87 10
20 04 82 47 65
19 01 23 75 03 34
88 02 77 73 07 63 67
99 65 04 28 06 16 70 92
41 41 26 56 83 40 80 70 33
41 48 72 33 47 32 37 16 94 29
53 71 44 65 25 43 91 52 97 51 14
70 11 33 28 77 73 17 78 39 68 17 57
91 71 52 38 17 14 91 43 58 50 27 29 48
63 66 04 68 89 53 67 30 73 16 69 87 40 31
04 62 98 27 23 09 70 98 73 93 38 53 60 04 23'''), delim_whitespace=True, header=None).fillna(0).values.astype('int')

A_cum = A.copy()
A_bool = np.zeros(A.shape, dtype="int")
for i in np.arange(len(A)-1, 0, -1):
    A_bool[i - 1, :-1] = (np.diff(A_cum[i]) > 0).astype("int")
    A_cum[i - 1] += A_cum[i, np.arange(len(A)) + A_bool[i - 1]]
print(A_cum[0, 0])

1074
CPU times: user 4.81 ms, sys: 1.56 ms, total: 6.37 ms
Wall time: 5.19 ms


In [20]:
%%time
## Problem 19: Counting Sundays

date_begin = "1901-01-01"
date_end = "2000-12-31"

S = pd.Series(pd.date_range(date_begin, date_end, freq="MS").strftime('%A'))
print(S.value_counts()["Sunday"])

171
CPU times: user 80.6 ms, sys: 1.96 ms, total: 82.5 ms
Wall time: 81.1 ms


In [21]:
%%time
## Problem 20: Factorial digit sum

n = 100

print(np.array(list(str(special.factorial(n, exact=True))), dtype="int").sum())

648
CPU times: user 321 µs, sys: 195 µs, total: 516 µs
Wall time: 375 µs


In [22]:
%%time
## Problem 21: Amicable numbers

n_max = 10000

def sum_divisors(x):
    return(sum(sympy.divisors(x)[:-1]))

S = pd.Series([sum_divisors(i) for i in np.arange(n_max)])
print(S.index.values[(S.index == S.loc[S.values]) & (S.index != S.values)].sum())

31626
CPU times: user 269 ms, sys: 5.58 ms, total: 274 ms
Wall time: 271 ms


In [23]:
%%time
## Problem 22: Names scores

df = pd.read_csv("https://projecteuler.net/project/resources/p022_names.txt").T

S = df.sort_index().reset_index()["index"]
print((S.apply(lambda x: sum([ord(i) - 64 for i in list(x)])) * (S.index.values + 1)).sum())

871198282
CPU times: user 1.86 s, sys: 39.2 ms, total: 1.89 s
Wall time: 3.01 s


In [24]:
%%time
## Problem 23: Non-abundant sums

n_max = 28123

def sum_divisors(x):
    return(sum(sympy.divisors(x)[:-1]))

S = pd.Series([sum_divisors(i) for i in np.arange(n_max)])
n_list = S.index.values[S.index < S.values]
print(S.index.difference(np.unique(np.add.outer(n_list, n_list))).values.sum())

4179871
CPU times: user 2.84 s, sys: 332 ms, total: 3.17 s
Wall time: 3.18 s


In [25]:
%%time
## Problem 24: Lexicographic permutations

n = 10
n_th = int(1e6)

n_list = list(range(n))
out = 0
n_th -= 1
for i in reversed(range(n)):
    i_val = special.factorial(i, exact=True)
    out += n_list.pop(n_th // i_val) * 10 ** i
    n_th %= i_val
print(out)

2783915460
CPU times: user 248 µs, sys: 115 µs, total: 363 µs
Wall time: 296 µs


In [26]:
%%time
## Problem 25: 1000-digit Fibonacci number

n = 1000

i = 3
i_val = 2
phi = (1 + np.sqrt(5)) / 2
print(int(np.ceil((n - 1 - np.log10(i_val)) / np.log10(phi) + i)))

4782
CPU times: user 255 µs, sys: 173 µs, total: 428 µs
Wall time: 328 µs


In [27]:
%%time
## Problem 26: Reciprocal cycles

n_max = 1000

out = []
for i in range(1, n_max):
    out.append(list(sympy.cycle_length(lambda x: 10 * x % i, 1))[0][0])
print(np.argmax(out) + 1)

983
CPU times: user 110 ms, sys: 2.56 ms, total: 113 ms
Wall time: 111 ms


In [28]:
%%time
## Problem 27: Quadratic primes

a_bound = 1000

a_list = np.arange(-a_bound, a_bound + 1)
A_bool = np.ones((len(a_list), len(a_list)), dtype="bool")

for n in itertools.count():
    A = n ** 2 + np.add.outer(n * a_list, a_list)
    A_update = [prime.isprime(i) for i in A[A_bool]]
    if np.any(A_update) == False:
        break
    A_bool[A_bool] = A_update
print(np.prod([a_list[i[0]] for i in np.where(A_bool)]))

-59231
CPU times: user 25.4 s, sys: 940 ms, total: 26.3 s
Wall time: 26.3 s


In [29]:
%%time
## Problem 28: Number spiral diagonals

n = 1001

print(4 * np.cumsum(np.insert(5 + np.arange((n + 1) // 2 - 1) * 8, 0, 1)).sum() - 3)

669171001
CPU times: user 559 µs, sys: 495 µs, total: 1.05 ms
Wall time: 570 µs


In [30]:
%%time
## Problem 29: Distinct powers

n_min = 2
n_max = 100

n_list = np.arange(n_min, n_max + 1, dtype="float")
print(len(np.unique(np.power.outer(n_list, n_list))))

9183
CPU times: user 1.68 ms, sys: 958 µs, total: 2.64 ms
Wall time: 1.38 ms


In [31]:
%%time
## Problem 30: Digit fifth powers

n = 5

n_list = np.arange(2, 9 ** n * n)
S = pd.Series(functools.reduce(np.add, [(n_list // (10 ** i) % 10) ** n for i in np.arange(n + 1)]), n_list)
print(sum(S.index[S == S.index].values))

443839
CPU times: user 59.7 ms, sys: 8.11 ms, total: 67.8 ms
Wall time: 66.3 ms


In [32]:
%%time
## Problem 31: Coin sums

n = 200
n_list = [1, 2, 5, 10, 20, 50, 100, 200]

def count_coin(x, val):
    if x == 0:
        return([[val]])
    out = []
    for i in range(val // n_list[x] + 1):
        out += [j + [i] for j in count_coin(x - 1, val - n_list[x] * i)]
    return(out)
print(len(count_coin(len(n_list) - 1, 200)))

73682
CPU times: user 423 ms, sys: 20 ms, total: 443 ms
Wall time: 440 ms


In [33]:
%%time
## Problem 32: Pandigital products

n_list = np.arange(1, 10)

out = []
for i in n_list:
    for j in n_list:
        res_list = np.setdiff1d(n_list, [i, j, (i * j) % 10])
        if len(res_list) == 6:
            for k in itertools.permutations(res_list, 3):
                k_val_0 = (i + k[0] * 10 + k[1] * 100 + k[2] * 1000) * j
                k_val_1 = (i + k[0] * 10 + k[1] * 100) * (j + k[2] * 10)
                prod_list = list(np.setdiff1d(res_list, k))
                if list(np.sort([int(x) for x in str(k_val_0)[:-1]])) == prod_list:
                    out.append(k_val_0)
                if list(np.sort([int(x) for x in str(k_val_1)[:-1]])) == prod_list:
                    out.append(k_val_1)
print(sum(np.unique(out)))

45228
CPU times: user 444 ms, sys: 10 ms, total: 454 ms
Wall time: 447 ms


In [34]:
%%time
## Problem 33: Digit cancelling fractions

out = []
for i in range(1, 10):
    for j in range(1, i):
        i_val = sympy.Rational(j, i)
        for k in range(1, 10):
            if sympy.Rational(10 * k + j, 10 * i + k) == i_val:
                out.append(i_val)
            if sympy.Rational(10 * j + k, 10 * k + i) == i_val:
                out.append(i_val)
print(sympy.prod(out).q)

100
CPU times: user 12.3 ms, sys: 575 µs, total: 12.8 ms
Wall time: 12.4 ms


In [35]:
%%time
## Problem 34: Digit factorials

n_min = 10

n_max = special.factorial(9, exact=True)
n_dict = {str(i): special.factorial(i, exact=True) for i in range(10)}
print(sum(i for i in range(n_min, n_max) if sum(n_dict[j] for j in str(i)) == i))

40730
CPU times: user 606 ms, sys: 4.27 ms, total: 611 ms
Wall time: 608 ms


In [36]:
%%time
## Problem 35: Circular primes

n_max = int(1e6)

prime_set = set(prime.primerange(n_max))
out = []
for i in prime_set:
    i_len = 1 + int(np.log10(i))
    i_val = set(int((str(i) * 2)[j:(j+i_len)]) for j in range(i_len))
    if i_val.issubset(prime_set):
        out.append(i)
print(len(out))

55
CPU times: user 2.26 s, sys: 5.52 ms, total: 2.26 s
Wall time: 2.26 s


In [37]:
%%time
## Problem 36: Double-base palindromes

n_max = 1000

out = []
for i in range(1, n_max):
    i_val_0 = int(str(i)[:-1] + str(i)[::-1])
    i_val_1 = int(str(i) + str(i)[::-1])
    i_val_0_base2 = "{0:b}".format(i_val_0)
    i_val_1_base2 = "{0:b}".format(i_val_1)
    if i_val_0_base2 == i_val_0_base2[::-1]:
        out.append(i_val_0)
    if i_val_1_base2 == i_val_1_base2[::-1]:
        out.append(i_val_1)
print(sum(out))

872187
CPU times: user 4.81 ms, sys: 213 µs, total: 5.03 ms
Wall time: 4.92 ms


In [38]:
%%time
## Problem 37: Truncatable primes

n_stop = 11

n_max = int(1e9)
n_list = prime.primerange(n_max)[4:]
out = []
left_list = {2, 3, 5, 7}
right_list = {2, 3, 5, 7}
for i in n_list:
    if i // 10 in left_list:
        left_list.add(i)
    if int(str(i)[1:]) in right_list:
        right_list.add(i)
        if i // 10 in left_list:
            out.append(i)
            if len(out) >= n_stop:
                break
print(sum(out))

748317
CPU times: user 5.1 s, sys: 766 ms, total: 5.87 s
Wall time: 5.87 s


In [39]:
%%time
## Problem 38: Pandigital multiples

n_list = np.arange(1, 10)

out = []
for i in [4]:
    for j in itertools.permutations(n_list, i):
        res_list = np.setdiff1d(n_list, j)
        i_val = int("".join([str(k) for k in j]))
        i_val_list = [int(k) for k in str(2 * i_val)]
        if list(np.sort(i_val_list)) == list(res_list):
            out.append(int("".join([str(k) for k in list(j) + i_val_list])))
print(max(out))

932718654
CPU times: user 280 ms, sys: 3.04 ms, total: 283 ms
Wall time: 281 ms


In [40]:
%%time
## Problem 39: Integer right triangles

n_max = 1000
n_min = 10

n_list = np.arange(n_min, n_max + 1)
out = []
for i in n_list:
    a_list = np.arange(1, i // 2)
    A = (i - np.add.outer(a_list, a_list)) ** 2 == np.add.outer(a_list ** 2, a_list ** 2)
    out.append(np.sum(A))
print(n_list[np.argmax(out)])

840
CPU times: user 700 ms, sys: 344 ms, total: 1.04 s
Wall time: 1.04 s


In [41]:
%%time
## Problem 39: Integer right triangles
### Alternative solve 1

n_max = 1000
n_min = 10

@numba.jit
def right_triangle_solve(p, a, b):
    return(a ** 2 + b ** 2 - (p - a - b) ** 2)

n_list = np.arange(n_min, n_max + 1)
out = []
for i in n_list:
    b = (i - 1) // 2
    count = 0
    for a in np.arange(1, i // 2):
        b = round(optimize.brentq(lambda x: right_triangle_solve(i, a, x), b - 3, b + 1, xtol = 0.5))
        if right_triangle_solve(i, a, b) == 0:
            count += 1
    out.append(count)
print(n_list[np.argmax(out)])

840
CPU times: user 1.91 s, sys: 124 ms, total: 2.03 s
Wall time: 2.17 s


In [42]:
%%time
## Problem 40: Champernowne's constant

n_max = int(1e6)
n_list = [1, 10, 100, 1000, 10000, 100000, 1000000]

s = "".join([str(i) for i in range(n_max)])
print(np.prod([int(s[i]) for i in n_list]))

210
CPU times: user 348 ms, sys: 33.3 ms, total: 381 ms
Wall time: 378 ms


In [43]:
%%time
## Problem 41: Pandigital prime

n_list = range(1, 10)
out = []
for i in n_list:
    for j in itertools.permutations(range(1, i + 1)):
        i_val = int("".join([str(k) for k in j]))
        if prime.isprime(i_val):
            out.append(i_val)
print(max(out))

7652413
CPU times: user 4.25 s, sys: 117 ms, total: 4.36 s
Wall time: 4.29 s


In [44]:
%%time
## Problem 42: Coded triangle numbers

url = requests.get("https://projecteuler.net/project/resources/p042_words.txt")

out = np.bincount([sum(ord(j) - 64 for j in i[1:-1]) for i in url.text.split(",")])
n_list = np.arange(1, np.sqrt(2 * len(out)) + 1, dtype = "int")
n_list = n_list * (n_list + 1) // 2
print(sum(out[n_list[n_list < len(out)]]))

162
CPU times: user 27.3 ms, sys: 6.34 ms, total: 33.7 ms
Wall time: 538 ms


In [45]:
%%time
## Problem 43: Sub-string divisibility

n_list = range(10)
a_list = [2, 3, 5, 7, 11, 13, 17]

print(sum([int("".join(str(j) for j in i)) for i in itertools.permutations(n_list) 
     if all((i[j + 1] * 100 + i[j + 2] * 10 + i[j + 3]) % j_val == 0 for j, j_val in enumerate(a_list))]))

16695334890
CPU times: user 6.82 s, sys: 6.83 ms, total: 6.83 s
Wall time: 6.83 s


In [46]:
%%time
## Problem 44: Pentagon numbers

def pentagonal(x):
    return(x * (3 * x - 1) // 2)

def pentagonal_solve():
    n_list = set()
    a_list = []
    for i in itertools.count(1):
        i_val = pentagonal(i)
        n_list.add(pentagonal(2 * i))
        n_list.add(pentagonal(2 * i + 1))
        for j in a_list:
            if ((i_val - j) in n_list) & ((2 * i_val - j) in n_list):
                return(j)
        a_list.append(i_val)
        
print(pentagonal_solve())

5482660
CPU times: user 466 ms, sys: 3.98 ms, total: 470 ms
Wall time: 467 ms


In [47]:
%%time
## Problem 45: Triangular, pentagonal, and hexagonal

n_min = 285

n_list_pen = set()
n_list_hex = set()
for i in itertools.count(2):
    i_val = i * (i + 1) // 2
    n_list_pen.add(i * (3 * i - 1) // 2)
    n_list_hex.add(i * (2 * i - 1))
    if (i > n_min) & (i_val in n_list_pen) & (i_val in n_list_hex):
        break
print(i_val)

1533776805
CPU times: user 79.4 ms, sys: 3.98 ms, total: 83.3 ms
Wall time: 81.6 ms


In [48]:
%%time
## Problem 46: Goldbach's other conjecture

for i in itertools.count(1):
    i_val = 2 * i + 1
    if not sympy.isprime(i_val):
        i_choice = np.sqrt((i_val - prime.primerange(i_val)) / 2)
        if (i_choice.round(0) != i_choice).all():
            break
print(i_val)

5777
CPU times: user 96.2 ms, sys: 11.5 ms, total: 108 ms
Wall time: 99.3 ms


In [49]:
%%time
## Problem 47: Distinct primes factors

n = 4

count = 0
for i in itertools.count():
    i_val = len(sympy.primefactors(i))
    if i_val == n:
        count += 1
        if count >= n:
            break
    else:
        count = 0
print(i - 3)

134043
CPU times: user 3.16 s, sys: 5.75 ms, total: 3.17 s
Wall time: 3.17 s


In [50]:
%%time
## Problem 48: Self powers

n_max = 1000
n = 10

def power_last_digits(x, n):
    out = 1
    a = int(10 ** n)
    for i in range(x):
        out *= x
        out %= a
    return(out)
print(sum(power_last_digits(i, n) for i in 1 + np.arange(n_max)) % int(10 ** n))

9110846700
CPU times: user 146 ms, sys: 1.91 ms, total: 148 ms
Wall time: 146 ms


In [51]:
%%time
## Problem 49: Prime permutations

n = 3
n_min = 1000
n_max = 10000
n_exclude = 4817

def prime_perm():
    n_list = prime.primerange(n_max)
    n_list = n_list[n_list > n_min]
    S = pd.Series([tuple(np.sort(list(str(i)))) for i in n_list], n_list, name="perm")
    for i_key, i in S.reset_index().groupby("perm")["index"]:
        if len(i) >= 3:
            i_val = np.abs(np.add.outer(i, -i))
            for j in range(len(i_val)):
                if (len(np.unique(i_val[j])) < len(i_val)) & (i.values[j] != n_exclude):
                    j_val = pd.Series(i_val[j]).duplicated(keep = False)
                    return(int("".join(str(i.values[k]) for k in [j_val.index[j_val][0], j, j_val.index[j_val][1]])))
print(prime_perm())

296962999629
CPU times: user 60 ms, sys: 4.22 ms, total: 64.3 ms
Wall time: 61.1 ms


In [52]:
%%time
## Problem 50: Consecutive prime sum

n_max = int(1e6)

n_list = prime.primerange(n_max)
out = np.zeros(n_max)
for i in range(len(n_list)):
    i_val = n_list[i:].cumsum()
    i_val = i_val[i_val < n_max]
    out[i_val] = np.maximum(out[i_val], 1 + np.arange(len(i_val)))
print(n_list[np.argmax(out[n_list])])

997651
CPU times: user 12.3 s, sys: 138 ms, total: 12.5 s
Wall time: 13.2 s


In [53]:
%%time
## Problem 51: Prime digit replacements

len_min = 8

for n in itertools.count(2):
    n_list = prime.primerange(10 ** n)
    n_list = n_list[n_list > 10 ** (n - 1)]
    df = pd.DataFrame([[int(j) for j in str(i)] for i in n_list], n_list)
    out = []
    for i in range(1, n):
        for j in itertools.combinations(np.arange(1, n), i):
            j_val = np.setdiff1d(np.arange(n), j)
            if len(j_val) == 1:
                for k_key, k in df.groupby(j):
                    if len(k) >= len_min:
                        out.append(min(k.index.values))
            else:
                for k_key, k in df.groupby(j):
                    k_val = (np.diff(k.values[:, j_val], axis=1) == 0).all(axis=1)
                    if sum(k_val) >= len_min:
                        out.append(min(k.index.values[k_val]))
    if out:
        break
print(min(out))

121313
CPU times: user 17.3 s, sys: 176 ms, total: 17.5 s
Wall time: 17.5 s


In [54]:
%%time
## Problem 52: Permuted multiples

n = 6

def contain_digit(x):
    return(np.sort(list(str(x))))

for i in itertools.count(1):
    i += 10 ** len(str(i))
    i_val = contain_digit(i)
    if all(np.array_equal(i_val, contain_digit(j * i)) for j in np.arange(2, n)):
        break
print(i)

142857
CPU times: user 1.92 s, sys: 6.98 ms, total: 1.92 s
Wall time: 1.92 s


In [55]:
%%time
## Problem 53: Combinatoric selections

n_max = 100
n_val_max = int(1e6)

out = []
for i in 1 + np.arange(n_max):
    for j in np.arange(i + 1):
        j_val = special.comb(i, j)
        if j_val > n_val_max:
            j = j * 2 - 1
            break
    out.append(i - j)
print(sum(out))

4075
CPU times: user 11.9 ms, sys: 1.57 ms, total: 13.4 ms
Wall time: 19.1 ms


In [56]:
%%time
## Problem 54: Poker hands

n = 5
n_dict = {"2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9, "T": 10, "J": 11, "Q": 12, "K": 13, "A": 14}
def poker_rank(df):
    df_rank = pd.DataFrame([np.bincount(i, minlength = 5) for i_key, i in df.iterrows()])
    df_rank[5] = (df_rank[1] == 5) & ((df.diff(axis=1) == 1).sum(axis=1) == 1)
    df_rank = 7 * df_rank[4] + 2 * (df_rank[3] & df_rank[2]) + 4 * df_rank[5] + 3 * df_rank[3] + df_rank[2] + (df + np.arange(15) / 100).idxmax(axis=1) / 100
    return(df_rank)

df = pd.read_fwf("https://projecteuler.net/project/resources/p054_poker.txt", header=None)
df_1 = pd.DataFrame([np.bincount([n_dict[j[0]] for j in i], minlength = 15) for i_key, i in df.iloc[:, :n].iterrows()])
df_2 = pd.DataFrame([np.bincount([n_dict[j[0]] for j in i], minlength = 15) for i_key, i in df.iloc[:, n:].iterrows()])
S_suit = df.iloc[:, :n].apply(lambda x: len(set(i[1] for i in x)) == 1, axis=1).astype('int') - df.iloc[:, n:].apply(lambda x: len(set(i[1] for i in x)) == 1, axis=1).astype('int')
df_rank_1 = poker_rank(df_1)
df_rank_2 = poker_rank(df_2)
print(np.sum(df_rank_1 - df_rank_2 + 5 * S_suit > 0))

376
CPU times: user 478 ms, sys: 5.61 ms, total: 484 ms
Wall time: 1.52 s


In [57]:
%%time
## Problem 55: Lychrel numbers

n_max = 10000
max_iter = 50

n_dict = {}
out = np.ones(n_max, dtype="bool")
for i in range(n_max):
    i_val = i
    for j in range(max_iter):
        i_val += int(str(i_val)[::-1])
        if int(str(i_val)[::-1]) == i_val:
            out[i] = 0
            break
print(sum(out))

249
CPU times: user 121 ms, sys: 2.43 ms, total: 123 ms
Wall time: 121 ms


In [58]:
%%time
## Problem 56: Powerful digit sum

n_max = 100
out = []
for i in range(n_max):
    i_val = 1
    out.append([])
    for j in range(n_max):
        i_val *= i
        out[-1].append(sum([int(k) for k in str(i_val)]))     
print(np.max(out))

972
CPU times: user 253 ms, sys: 3.57 ms, total: 256 ms
Wall time: 256 ms


In [59]:
%%time
## Problem 57: Square root convergents

n_max = 1000

n_list_0 = [3]
n_list_1 = [2]
for i in range(n_max):
    n_list_1.append(n_list_0[-1] + n_list_1[-1])
    n_list_0.append(n_list_1[-1] + n_list_1[-2])
print(sum([len(str(i)) > len(str(j)) for i, j in zip(n_list_0, n_list_1)]))

153
CPU times: user 3.54 ms, sys: 129 µs, total: 3.67 ms
Wall time: 3.61 ms


In [60]:
%%time
## Problem 58: Spiral primes

r_min = 0.1

i_val = 0
for i in itertools.count(1):
    i_val += sum(prime.isprime(j) for j in (2 * i + 1) ** 2 + 2 * i * (np.arange(4) - 3))
    if i_val / (4 * i + 1) < r_min:
        break
print(2 * i + 1)

26241
CPU times: user 819 ms, sys: 17.4 ms, total: 837 ms
Wall time: 827 ms


In [61]:
%%time
## Problem 59: XOR decryption

n = 3
url = requests.get("https://projecteuler.net/project/resources/p059_cipher.txt")

S = pd.Series(url.text.split(","), dtype="uint8")
grp = S.groupby(S.index.values % n)
key = np.bitwise_xor(np.array([pd.value_counts(i).index[0] for i_key, i in grp], "uint8"), ord(" "))
print(sum([sum(np.bitwise_xor(i.values, key[i_key])) for i_key, i in grp]))

107359
CPU times: user 27.7 ms, sys: 4.82 ms, total: 32.5 ms
Wall time: 652 ms


In [62]:
%%time
## Problem 60: Prime pair sets

n = 5
n_max = int(1e8)
n_list = [list() for i in range(n)]
prime_set = set(prime.primerange(n_max))
for i in itertools.count(1):
    i_val = sympy.prime(i)
    for j in range(n - 1):
        for k in n_list[j]:
            if all(int("{}{}".format(l[0], l[1])) in prime_set for l in itertools.permutations(k + [i_val], 2)):
                n_list[j + 1].append(k + [i_val])
    if n_list[n - 1]:
        break
    n_list[0].append([i_val])
print(sum(n_list[n - 1][0]))

26033
CPU times: user 50.9 s, sys: 499 ms, total: 51.4 s
Wall time: 51.6 s


In [63]:
%%time
## Problem 61: Cyclical figurate numbers

n_len = 6
n_min = 1000
n_max = 10000

n = np.arange(2 * np.sqrt(n_max), dtype="int")
n_list = [i[(i < n_max) & (i >= n_min) & (i % 100 >= 10)] for i in [n * (n + 1) // 2, n ** 2, n * (3 * n - 1) // 2, n * (2 * n - 1), n * (5 * n - 3) // 2, n * (3 * n - 2)]]
df_list = [pd.DataFrame([i // 100, i % 100]).T for i in n_list]
for i in itertools.permutations(range(n_len - 1)):
    df = df_list[n_len - 1]
    for j_key, j in enumerate(i):
        df = df.set_index(j_key + 1).join(df_list[j].rename(columns=lambda x: x + j_key + 1).set_index(j_key + 1), lsuffix="old", how="inner").reset_index().rename(columns={"index": j_key + 1})
    i_val = df[n_len] == df[0]
    if i_val.any():
        break
print((df.loc[i_val, range(n_len)].values * (100 + 1)).sum())

28684
CPU times: user 1.74 s, sys: 20.6 ms, total: 1.76 s
Wall time: 1.77 s


In [64]:
%%time
## Problem 62: Cubic permutations

out = []
for n in itertools.count(1):
    n_list = np.arange(10 ** (n - 1), 10 ** n) ** 3
    df = pd.DataFrame(np.vstack(np.bincount([int(j) for j in i], minlength=10) for i in n_list.astype("str")), n_list)
    out += [i_val for i, i_val in df.groupby(list(range(10))) if len(i_val) >= 5]
    if out:
        break
print(pd.concat(out).index.min())

127035954683
CPU times: user 1.27 s, sys: 33.9 ms, total: 1.3 s
Wall time: 1.28 s


In [65]:
%%time
## Problem 63: Powerful digit counts

n_max = 100
n_list = np.arange(n_max)
print(((np.multiply.outer(np.log10(np.arange(1, 10)), n_list) + 1).astype("int") == n_list).sum())

49
CPU times: user 394 µs, sys: 190 µs, total: 584 µs
Wall time: 469 µs


In [66]:
%%time
## Problem 64: Odd period square roots

n_max = 10000

def sqrt_fraction_period(x):
    x_sqrt = np.sqrt(x)
    if x_sqrt == int(x_sqrt):
        return(0)
    x_q = [x - int(x_sqrt) ** 2]
    x_m = [(x_sqrt + int(x_sqrt)) // x_q[-1]]
    x_r = [x_m[-1] * x_q[-1] - int(x_sqrt)]
    for i in itertools.count():
        x_q.append((x - x_r[-1] ** 2) // x_q[-1])
        x_m.append((x_sqrt + x_r[-1]) // x_q[-1])
        x_r.append(x_m[-1] * x_q[-1] - x_r[-1])
        if (x_q[-1] == x_q[0]) & (x_m[-1] == x_m[0]) & (x_r[-1] == x_r[0]):
            break
    return(len(x_m) - 1)

print(sum([sqrt_fraction_period(i) % 2 == 1 for i in range(n_max + 1)]))

1322
CPU times: user 618 ms, sys: 2.36 ms, total: 620 ms
Wall time: 619 ms


In [67]:
%%time
## Problem 65: Convergents of e

n = 100

def continued_fraction(m, x):
    x_q = [0, 1]
    for i in reversed(x):
        x_q.append(i * x_q[-1] + x_q[-2])
    x_q.append(m * x_q[-1] + x_q[-2])
    return(x_q)

n_list = np.hstack([np.ones([n // 3 + 1, 1]), 2 * (1 + np.arange(n // 3 + 1))[:, np.newaxis], np.ones([n // 3 + 1, 1])]).flatten()[:n - 1]
print(sum([int(j) for j in str(continued_fraction(2, [int(i) for i in n_list])[-1])]))

272
CPU times: user 497 µs, sys: 535 µs, total: 1.03 ms
Wall time: 593 µs


In [109]:
## Problem 66: Diophantine equation
## integer method

n_max = 1000

n_list = np.arange(1 + n_max, dtype="uint")
out = np.zeros(1 + n_max, dtype="uint")
n_list = n_list[np.sqrt(n_list) % 1 != 0]
n_thres = int(1e9)
n_batch = int(1e5)

y = 2 + np.arange(n_batch, dtype="uint")
y2 = np.empty(n_batch, dtype="uint")
Y2D = np.empty([n_batch, len(n_list)], dtype="uint")
X = np.empty([n_batch, len(n_list)], dtype="longdouble")
X2 = np.empty([n_batch, len(n_list)], dtype="uint")
X_bool = np.empty([n_batch, len(n_list)], dtype="bool")
for i in np.arange(2, n_thres, n_batch):
    np.square(y, y2)
    np.outer(y2, n_list, Y2D)
    Y2D += 1
    np.sqrt(Y2D, X)
    np.square(X.round().astype("uint"), X2)
    np.equal(X2, Y2D, X_bool)
    i_val = X_bool.any(axis=0)
    if i_val.any():
        out[n_list[i_val]] = X.round().astype("uint")[X_bool[:, i_val].argmax(axis=0), i_val]
        n_list = n_list[~i_val]
        print(y_list[-1], len(n_list), 2 * np.log2(y_list[-1]) + np.log2(n_list[-1]))
        if len(n_list) == 0:
            break
        Y2D = np.empty([n_batch, len(n_list)], dtype="uint")
        X = np.empty([n_batch, len(n_list)], dtype="longdouble")
        X2 = np.empty([n_batch, len(n_list)], dtype="uint")
        X_bool = np.empty([n_batch, len(n_list)], dtype="bool") 
    y_list += n_batch
np.argmax(out)

100001 330 43.1850940873
200001 311 45.1850796605
300001 292 46.3549998529
400001 280 47.185072447
500001 273 47.8289271941
600001 270 48.354995044
700001 266 48.7997791996
800001 264 49.1850688403
900001 263 49.5249184424
1000001 259 49.8289243087
1100001 257 50.1039310939
1300001 255 50.5845034725
1400001 253 50.7983337218
1500001 252 50.9974049315
1600001 251 51.18362362
1800001 250 51.5234734225
1900001 248 51.6794783622
2000001 247 51.8274794491
2100001 245 51.9682580362
2200001 243 52.1024863655
2400001 242 52.3535480204
2500001 240 52.4713353504
2800001 239 52.7983326913
3000001 238 52.9974039697
3200001 236 53.1836227184
3300001 234 53.2709660673
3700001 233 53.6010844653
4100001 232 53.8972816672
4200001 230 53.9668124868
4600001 229 54.2293014936
4800001 228 54.3521025568
5000001 227 54.4698899108
5100001 225 54.5270282039
5300001 224 54.6380184078
6400001 223 55.1821774051
6500001 222 55.2269130242
6600001 220 55.2709656301
7100001 219 55.4816715996
7300001 217 55.5618264672

KeyboardInterrupt: 

In [68]:
%%time
## Problem 67: Maximum path sum II

A = pd.read_fwf("https://projecteuler.net/project/resources/p067_triangle.txt", delim_whitespace=True, header=None).fillna(0).values.astype('int')

A_cum = A.copy()
A_bool = np.zeros(A.shape, dtype="int")
for i in np.arange(len(A)-1, 0, -1):
    A_bool[i - 1, :-1] = (np.diff(A_cum[i]) > 0).astype("int")
    A_cum[i - 1] += A_cum[i, np.arange(len(A)) + A_bool[i - 1]]
print(A_cum[0, 0])

7273
CPU times: user 50.3 ms, sys: 9.02 ms, total: 59.4 ms
Wall time: 541 ms


In [69]:
%%time
## Problem 68: Magic 5-gon ring

n_max = 10
n_list = np.arange(1, n_max)

out = []
for i in itertools.permutations(n_list):
    i = list(i) + [n_max]
    i_val = np.array([[i[5 + j], i[j], i[(j + 1) % 5]] for j in range(5)])
    if (np.diff(i_val.sum(axis=1)) == 0).all():
        i_begin = np.argmin(i_val[:, 0])
        out.append(int("".join(["".join([str(k) for k in j]) for j in i_val[i_begin:].tolist() + i_val[:i_begin].tolist()])))
print(max(out))

6531031914842725
CPU times: user 11.2 s, sys: 12 ms, total: 11.2 s
Wall time: 11.2 s


In [70]:
%%time
## Problem 69: Totient maximum

n_max = 1000000

i_val = 1
out = []
for i in itertools.count(1):
    i_val *= prime.prime(i)
    if i_val > n_max:
        break
    out.append(i_val)
print(max(out))

510510
CPU times: user 468 µs, sys: 275 µs, total: 743 µs
Wall time: 608 µs


In [71]:
%%time
## Problem 70: Totient permutation

n_max = int(1e7)

prime_list = prime.primerange(2 * n_max)
log_prime_list = np.log(prime_list)
val_list = np.log(1 - 1 / prime_list)

def top_totient(n_max, prime_max):
    log_n_max = np.log(n_max)
    i_list = [0]
    i_val_list = [prime_list[i] for i in i_list]
    log_i_val_list = [log_prime_list[i] for i in i_list]
    out = sum(log_i_val_list)
    i = i_list[-1]
    i_bool = True
    while True:
        i += 1
        i_val = prime_list[i]
        log_i_val = log_prime_list[i]
        if i_val > prime_max:
            i_bool = False
        if i_bool:
            if out + log_i_val < log_n_max:
                i_list.append(i)
                out += log_i_val
                i_val_list.append(i_val)
                log_i_val_list.append(log_i_val)
                yield [out, val_list[i_list].sum()]
            else:
                i -= 1
                i_bool = False
        elif (i_val <= prime_max) & (out + log_i_val - log_i_val_list[-1] < log_n_max):
            i_list[-1] = i
            out += log_i_val - log_i_val_list[-1]
            i_val_list[-1] = prime_list[i]
            log_i_val_list[-1] = log_prime_list[i]
            yield [out, val_list[i_list].sum()]
        else:
            i_list.pop()
            if len(i_list) == 0:
                break
            i_list[-1] += 1
            i = i_list[-1]
            i_val_list.pop()
            i_val_list[-1] = prime_list[i]
            log_i_val_list.pop()
            log_i_val_list[-1] = log_prime_list[i]
            out = sum(log_i_val_list)
            yield [out, val_list[i_list].sum()]
            i_bool = True
            
df = pd.DataFrame(np.exp(list(top_totient(n_max, n_max))), columns=["n", "phi"])
df = df.sort_values(["phi"], ascending=False).reset_index(drop=True).assign(n = lambda x: x["n"], phi = lambda x: (x["n"] * x["phi"])).round().astype('int')
for i, i_phi in df.values:
    i_val = np.bincount([int(j) for j in str(i)], minlength=10)
    i_val_phi = np.bincount([int(j) for j in str(i_phi)], minlength=10)
    if all(i_val_phi == i_val):
        break
print(i)

8319823
CPU times: user 1min 33s, sys: 1.52 s, total: 1min 35s
Wall time: 1min 35s


In [72]:
%%time
## Problem 71: Ordered fractions

n_max = int(1e6)
p = 3
q = 7

q_list = 1 + np.arange(n_max)
p_list = (q_list * p / q).round().astype('int')
r_list = p_list * q / (q_list * p) - 1
p_list = p_list[r_list < 0]
q_list = q_list[r_list < 0]
r_list = p_list * q / (q_list * p) - 1
print(p_list[np.argmax(r_list)])

428570
CPU times: user 45.8 ms, sys: 26.3 ms, total: 72.2 ms
Wall time: 69.5 ms


In [73]:
%%time
## Problem 72: Counting fractions

n_max = int(1e6)

prime_list = prime.primerange(2 * n_max)
log_prime_list = np.log(prime_list)
log_phi_n_list = np.log(1 - 1 / prime_list)

def full_totient(n_max, prime_max):
    log_n_max = np.log(n_max + 0.5)
    i_list = []
    i_val_list = []
    log_i_val_list = []
    out = 0
    
    i = 0
    i_val = prime_list[i]
    log_i_val = log_prime_list[i]
    i_bool = True
    while True:
        i_val = prime_list[i]
        log_i_val = log_prime_list[i]
        if i_val > prime_max:
            i_bool = False
        if i_bool:
            if out + log_i_val < log_n_max:
                i_list.append(i)
                i_val_list.append(i_val)
                log_i_val_list.append(log_i_val)
                out += log_i_val
                yield [out, log_phi_n_list[list(set(i_list))].sum()]
            else:
                i += 1
                i_bool = False
        elif (i_val <= prime_max) & (out + log_i_val - log_i_val_list[-1] < log_n_max):
            i_list[-1] = i
            out += log_i_val - log_i_val_list[-1]
            i_val_list[-1] = prime_list[i]
            log_i_val_list[-1] = log_prime_list[i]
            yield [out, log_phi_n_list[list(set(i_list))].sum()]
            i += 1
        else:
            i_list.pop()
            if len(i_list) == 0:
                break
            i_list[-1] += 1
            i = i_list[-1]
            i_val_list.pop()
            i_val_list[-1] = prime_list[i]
            log_i_val_list.pop()
            log_i_val_list[-1] = log_prime_list[i]
            out = sum(log_i_val_list)
            yield [out, log_phi_n_list[list(set(i_list))].sum()]
            i_bool = True
            
df = pd.DataFrame(np.exp(list(full_totient(n_max, n_max))), columns=["n", "phi"])
df = df.assign(n = lambda x: x["n"], phi = lambda x: (x["n"] * x["phi"])).round().astype('int').sort_values("n")
print(sum(df["phi"]))

303963552391
CPU times: user 12.9 s, sys: 205 ms, total: 13.1 s
Wall time: 13.1 s


In [74]:
%%time
## Problem 73: Counting fractions in a range

n_max = 12000
i_max = 1 / 2
i_min = 1 / 3

n_list = 1 + np.arange(n_max)

out = [i for i in set(np.divide.outer(n_list[:int(n_max * i_max + 1)], n_list).flatten()) if i_min < i < i_max]
print(len(out))

7295372
CPU times: user 42.2 s, sys: 4.11 s, total: 46.4 s
Wall time: 47.1 s


In [75]:
%%time
## Problem 74: Digit factorial chains

n_max = int(1e6)
n_dict = {str(i): special.factorial(i, exact=True) for i in range(10)}
    
chain_val_dict = {}
out = []

def chain_path(x):
    x_val = x[-1]
    x_next = chain_val_dict.get(x_val)
    if x_next is None:
        x_next = sum(n_dict[i] for i in str(x_val))
        chain_val_dict[x_val] = x_next
    if x_next in x:
        return(x)
    else:
        x.append(x_next)
        return(chain_path(x))

for i in range(n_max):
    out.append(chain_path([i]))
print(sum(len(i) == 60 for i in out))

402
CPU times: user 33.2 s, sys: 648 ms, total: 33.9 s
Wall time: 33.9 s


In [76]:
%%time
## Problem 75: Singular integer right triangles

n_max = 1500000

n_list = 1 + np.arange(int(np.sqrt(n_max)))
m_list = 1 + np.arange(0, int(np.sqrt(n_max)), 2)
nm = np.outer(2 * n_list, m_list)

A = (nm + m_list ** 2).T
B = nm.T + 2 * n_list ** 2
C = A + 2 * n_list ** 2
L = A + B + C
S = pd.Series((C / L).flatten(), L.flatten()).sort_index().loc[:n_max].drop_duplicates().groupby(level=0).count()
out = np.zeros(n_max + 1, dtype="int")
out[S.index] = S.values
for i, i_val in S.iteritems():
    out[np.arange(2 * i, n_max + 1, i)] += i_val
print(sum(out == 1))

161667
CPU times: user 4.13 s, sys: 77.4 ms, total: 4.21 s
Wall time: 4.2 s


In [171]:
%%time
## Problem 76: Counting summations

n = 100

class CountSum():
    '''
    Problem 66 - 68
    '''
    def __init__(self, n_list = None):
        self.n_list = n_list
        self.val_dict = {}
        
    def count(self, x, val, key):
        if val == 0:
            return(1)
        out = self.val_dict.get((x, val, key))
        if out is None:
            diff = self.n_list[key + 1] - self.n_list[key]
            if key < (len(self.n_list) - 1):
                if val >= diff:
                    out = sum(self.count(i, val - i * diff, key + 1) for i in 1 + np.arange(min(x, val // diff)))
                else:
                    out = 0
                self.val_dict[(x, val, key)] = out
            else:
                out = int(x % diff == 0)
        return(out)
    
    def countint(self, x, val):
        if val == 0:
            return(1)
        out = self.val_dict.get((x, val))
        if out is None:
            out = sum(self.countint(i, val - i) for i in 1 + np.arange(min(x, val)))
            self.val_dict[(x, val)] = out
        return(out)
    
    def countsum(self, n):
        if self.n_list is None:
            return(sum(self.countint(i, n - i) for i in 1 + np.arange(n)))
        else:
            return(sum(self.count(i, n - self.n_list[0] * i, 0) for i in 1 + np.arange(n // self.n_list[0])))
        
counter = CountSum()
print(counter.countsum(n) - 1)

190569291
CPU times: user 63 ms, sys: 12.7 ms, total: 75.7 ms
Wall time: 66 ms


In [156]:
%%time
## Problem 77: Prime summations

n_min = 2
n_out = 5000

counter = CountSum(prime.prime_list)
for n in itertools.count(n_min):
    n_val = counter.countsum(n)
    if n_val > n_out:
        break
print(n)

71
CPU times: user 73.5 ms, sys: 12.5 ms, total: 86.1 ms
Wall time: 76.8 ms


In [172]:
## Problem 78: Coin partitions

n_q = int(1e6)

class CountSum():
    '''
    Problem 66 - 68
    '''
    def __init__(self, n_list = None):
        self.n_list = n_list
        self.val_dict = {}
        
    def count(self, x, val, key):
        if val == 0:
            return(1)
        out = self.val_dict.get((x, val, key))
        if out is None:
            diff = self.n_list[key + 1] - self.n_list[key]
            if key < (len(self.n_list) - 1):
                if val >= diff:
                    out = sum(self.count(i, val - i * diff, key + 1) for i in 1 + np.arange(min(x, val // diff)))
                else:
                    out = 0
                self.val_dict[(x, val, key)] = out
            else:
                out = int(x % diff == 0)
        return(out)
    
    def countint(self, x, val):
        if val == 0:
            return(1)
        out = self.val_dict.get((x, val))
        if out is None:
            out = sum(self.countint(i, val - i) for i in 1 + np.arange(min(x, val)))
            self.val_dict[(x, val)] = out % n_q
        return(out)
    
    def countsum(self, n):
        if self.n_list is None:
            return(sum(self.countint(i, n - i) for i in 1 + np.arange(n)))
        else:
            return(sum(self.count(i, n - self.n_list[0] * i, 0) for i in 1 + np.arange(n // self.n_list[0])))
    
counter = CountSum()
for n in itertools.count(1):
    n_val = counter.countsum(n)
    if n % 100 == 0:
        print(n, n_val)
    if n_val % n_q == 0:
        break
print(n)

100 141569292
200 2396029388
300 7058723602
400 14128041926
500 23845995027
600 36166553622
700 51107028659
800 68714911979
900 88779365430
1000 111481727991
1100 136673281283
1200 164276661657
1300 194410411672
1400 226932998261
1500 262106891117
1600 299587572183
1700 339505010007
1800 381896590849
1900 426575767364
2000 473970344166


KeyboardInterrupt: 

In [99]:
%%time
## Problem 79

S = pd.read_csv("https://projecteuler.net/project/resources/p079_keylog.txt", header=None, squeeze=True, dtype="str")

S = np.array([[int(j) for j in i] for i in S])
A = np.zeros([10, 10], dtype="int")
pair_list = np.vstack(S[:, i] for i in itertools.combinations(range(S.shape[1]), 2))
for i in pair_list:
    A[tuple(i)] += 1
    
n_list = np.nonzero(A.sum(axis=0) + A.sum(axis=1))[0].tolist()
A = A[np.ix_(n_list, n_list)]
out = []
while len(A):
    i_val = A.sum(axis=0).tolist().index(0)
    A = np.delete(A, i_val, axis=0)
    A = np.delete(A, i_val, axis=1)
    out.append(n_list.pop(i_val))
    
print(int("".join(str(i) for i in out)))

73162890
CPU times: user 17.2 ms, sys: 1.1 ms, total: 18.3 ms
Wall time: 411 ms


In [47]:
%%time
## Problem 80: Square root digital expansion

n_max = 100

out = []
for n in range(n_max + 1):
    if np.sqrt(n) % 1 != 0:
        out.append(sum(int(i) for i in str(sympy.N(sympy.sqrt(n), 10 + n_max)).replace(".", "")[:n_max]))
print(sum(out))

40886
CPU times: user 15.6 ms, sys: 0 ns, total: 15.6 ms
Wall time: 23.6 ms


In [3]:
## Problem 81: Path sum: two ways

A = pd.read_csv("https://projecteuler.net/project/resources/p081_matrix.txt", header=None).values
A.shape

(80, 80)